# Machine learning Grid Search

## Imports

In [1]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import pickle

warnings.filterwarnings('ignore')

sys.path.append('./..')
cufflinks.go_offline()

In [2]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, preprocessor
from auxiliar import parameters
from auxiliar.HtmlParser import HtmlParser

In [3]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
import copy

## Config

In [4]:
polarity_dim = 2
clasificadores=['lr', 'ls', 'mb', 'rf']
idf = True
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'machine_learning/tweeter/grid_search'

## Get data

In [5]:
cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
    cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
    cine = filter_corpus_small(cine.get_corpus())
elif polarity_dim == 5:
    cine = cine.get_corpus()
cine = cine[:5000]
used_data = pd.DataFrame(data_corpus)

#Intentando obtener datos del archivo csv...
./../Corpus/../data/general-corpus.csv
#Datos recuperados!


## Split data

In [6]:
split = used_data.shape[0] * 0.7
train_corpus = used_data.loc[:split - 1 , :]
test_corpus = used_data.loc[split:, :]

## Initialize ML

In [7]:
vect = vectorizerIdf if idf else vectorizer
ls = CalibratedClassifierCV(LinearSVC()) if polarity_dim == 2 else OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()))
lr = LogisticRegression(solver='lbfgs') if polarity_dim == 2 else OneVsRestClassifier(LogisticRegression())
mb = MultinomialNB() if polarity_dim == 2 else OneVsRestClassifier(MultinomialNB())
rf = RandomForestClassifier() if polarity_dim == 2 else OneVsRestClassifier(RandomForestClassifier())

In [8]:
pipeline_ls = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('ls', ls)
])
pipeline_lr = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('lr', lr)
])
pipeline_mb = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('mb', mb)
])
pipeline_rf = Pipeline([
    ('prep', copy.deepcopy(preprocessor)),
    ('vect', copy.deepcopy(vect)),
    ('rf', rf)
])

In [9]:
pipelines = {
    'ls': pipeline_ls,
    'lr': pipeline_lr,
    'mb': pipeline_mb,
    'rf': pipeline_rf
}
pipelines_train = {
    'ls': ls,
    'lr': lr,
    'mb': mb,
    'rf': rf
}

In [10]:
params = parameters.parameters_bin if polarity_dim == 2 else parameters.parameters

In [11]:
params

{'ls': {'ls__base_estimator__max_iter': [1000, 1200, 1500]},
 'lr': {'lr__max_iter': [100, 200, 500]},
 'mb': {'mb__alpha': [1.0, 0.5, 1.5]},
 'rf': {'rf__n_estimators': [10, 100]}}

## Train

In [12]:
folds = pd.read_pickle('./../data/pkls/folds.pkl') # k-folds precargados
folds = folds.values

In [13]:
pd.Series(train_corpus.content.values)

0                               @marodriguezb Gracias MAR
1       Off pensando en el regalito Sinde, la que se v...
2       Conozco a alguien q es adicto al drama! Ja ja ...
3       Toca @crackoviadeTV3 . Grabación dl especial N...
4       Buen día todos! Lo primero mandar un abrazo gr...
5       Desde el escaño. Todo listo para empezar #endi...
6       Bdías. EM no se ira de puente. Si vosotros os ...
7       Un sistema económico q recorta dinero para pre...
8                       #programascambiados caca d ajuste
9                                            Buen viernes
10      “@adri_22_22: #programascambiados es TT gracia...
11      La Universidad confía en De la Calle para enca...
12      ¿Me ayudáis a que #indultoneiro sea TT? Por si...
13      abcdesevilla.es: Recio no tiene «indicios pote...
14      abcdesevilla.es: Cuatro altos cargos de Empleo...
15      La marcha atrás del PP en posponer devolución ...
16      Viernes negro: Aumenta el paro en Noviembre y ...
17      Accide

In [14]:
results = {}
grids = {}
with tqdm(total=len(clasificadores) * 10) as pbar:
    for c in clasificadores:
        results[c] = { 'real': {}, 'predicted': {} }
        i = 0
        params[c].update(parameters.vect_params)
        params[c].update(parameters.prepro_params)
        param_grid = params[c]
        grid_search = GridSearchCV(pipelines[c], param_grid, verbose=2, scoring='accuracy', refit=False, cv=3)
        grid = grid_search.fit(train_corpus.content, train_corpus.polarity)
        grids[c] = grid
        best_parameters = grid.best_params_
        train_params = {}
        for param_name in sorted(parameters.vect_params.keys()):
            train_params.update({param_name[6:]: best_parameters[param_name]})
        vect.set_params(**train_params)
        preprocessor.set_params(**train_params)
        x_prepro = preprocessor.fit_transform(train_corpus.content)
        x_vect = vect.fit_transform(x_prepro, train_corpus.polarity).toarray()
        for train_index, test_index in folds:
            train_x = x_vect[train_index]
            train_y = train_corpus.polarity[train_index]
            test_x = x_vect[test_index]
            test_y = train_corpus.polarity[test_index]

            pipelines_train[c].fit(train_x, train_y)

            predicted = pipelines_train[c].predict(test_x)
            
            results[c]['real'][i] = test_y.values.tolist()
            results[c]['predicted'][i] = predicted.tolist()
            i = i + 1

            pbar.update(1)

    

  0%|          | 0/40 [00:00<?, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 
[CV]  lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=   1.6s
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV]  lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=   1.5s
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 
[CV]  lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=   1.5s
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   1.0s
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  lr__max_iter=100, prep__negation=0, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   1.0s
[CV] lr__max_iter=100, prep__negation=0, prep__repeated_letters=True

[CV]  lr__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   7.9s
[CV] lr__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  lr__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   9.5s
[CV] lr__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  lr__max_iter=500, prep__negation=1, prep__repeated_letters=True, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   8.9s


...

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  6.3min finished
 75%|███████▌  | 30/40 [17:56<00:56,  5.69s/it] [Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=  12.4s
[CV] prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.5s remaining:    0.0s


[CV]  prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=  12.3s
[CV] prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function stemize at 0x7f71cbddb0d0>, total=  12.6s
[CV] prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   8.5s
[CV] prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f71cbddb158> 
[CV]  prep__negation=0, prep__repeated_letters=True, rf__n_estimators=10, vect__tokenizer=<function tokenize at 0x7f71cbddb158>, total=   8.3s
[CV] prep__negation=0, prep__repeated_letters=T

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  5.4min finished
100%|██████████| 40/40 [25:02<00:00, 12.61s/it] 


In [15]:
results

{'lr': {'real': {0: [1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    0,
    0,
    0,
    1,
    1,
    0,
    1,
    0,
    0,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    0,
    0,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    0,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
   

In [16]:
pd.DataFrame(results).to_pickle('../results/'+name+'/'+base_dir+'/results-idf.pkl')

In [17]:
with open('../results/'+name+'/'+base_dir+'/grid_results-idf.pkl', 'wb') as fp:
    pickle.dump(grids, fp)

In [12]:
with open('../results/'+name+'/'+base_dir+'/grid_results-idf.pkl', 'rb') as fp:
    grids = pickle.load(fp)

In [14]:
test_results = {}
with tqdm(total=len(clasificadores)) as pbar:
    for c in clasificadores:
        test_results[c] = { 'real': {}, 'cine_real': {}, 'predicted': {}, 'cine_predicted': {} }
        i = 0
        grid = grids[c]
        best_parameters = grid.best_params_
        train_params = {}
        for param_name in sorted(parameters.vect_params.keys()):
            train_params.update({param_name[6:]: best_parameters[param_name]})
        vect.set_params(**train_params)
        vect.fit(data_corpus.content, data_corpus.polarity)
        x_vect = vect.transform(train_corpus.content).toarray()
        x_vect_test = vect.transform(test_corpus.content).toarray()
        x_vect_cine = vect.transform(cine.content).toarray()
        train_x = x_vect
        train_y = train_corpus.polarity
        test_x = x_vect_test
        test_y = test_corpus.polarity
        cine_y = cine.polarity

        pipelines_train[c].fit(train_x, train_y)

        predicted = pipelines_train[c].predict(test_x)
        cine_predicted = pipelines_train[c].predict(x_vect_cine)

        test_results[c]['real'][i] = test_y.values.tolist()
        test_results[c]['cine_real'][i] = cine_y.values.tolist()
        test_results[c]['predicted'][i] = predicted.tolist()
        test_results[c]['cine_predicted'][i] = cine_predicted.tolist()
        i = i + 1

        pbar.update(1)

  0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [19]:
pd.DataFrame(test_results).to_pickle('../results/'+name+'/'+base_dir+'/test_results-idf.pkl')